In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn import decomposition

import warnings
warnings.filterwarnings("ignore") 

seed = 433

In [2]:
data = pd.read_csv('311_2019.csv', delimiter=',')
data = data.rename(columns={'Incident Zip': 'Zip', 'Complaint Type' : 'Complaint'})
data = data[['Zip', 'Complaint']]
data.Zip=pd.to_numeric(data.Zip,errors='coerce')
data=data.loc[(data.Zip>=10000)&(data.Zip<11500)]
data.head()

,Zip,Complaint
1,11434.0,Damaged Tree
2,11212.0,Graffiti
3,10016.0,Graffiti
4,10032.0,Graffiti
5,11420.0,Noise - Commercial


In [3]:
data = data.groupby(['Zip', 'Complaint']).size().reset_index(name='count')
data.head()

,Zip,Complaint,count
0,10000.0,Abandoned Vehicle,1
1,10000.0,Animal in a Park,9
2,10000.0,Animal-Abuse,5
3,10000.0,Bike/Roller/Skate Chronic,4
4,10000.0,Consumer Complaint,23


In [4]:
#convert a list to a 2d table with zip codes as rows and complaint types as columns
data=pd.pivot_table(data,index='Zip',columns='Complaint',values='count',fill_value=0)
data.head()

Complaint,APPLIANCE,Abandoned Vehicle,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Animal-Abuse,Appliance,Asbestos,BEST/Site Safety,...,Vacant Lot,Vending,Violation of Park Rules,WATER LEAK,Water Conservation,Water Leak,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment
Zip,,,,,,,,,,,,,,,,,,,,,
10000.0,0,1,0,0,0,9,5,0,0,0,...,0,5,25,0,0,0,0,0,0,0
10001.0,0,9,19,0,0,7,16,0,1,0,...,0,64,8,0,2,0,2,23,0,0
10002.0,0,16,9,0,0,14,25,0,1,0,...,1,12,54,0,0,0,6,26,0,0
10003.0,1,18,7,1,0,12,50,0,8,0,...,0,29,43,0,1,0,17,25,0,0
10004.0,0,2,0,0,0,1,1,0,0,0,...,0,149,25,0,0,0,0,1,0,0


In [5]:
Total=data.sum(axis=1) #total 311 activity per zip code
data=data.div(data.sum(axis=1), axis=0) #normalize activity of various cathegories within zip code by total
data=data.loc[Total>100] #keep only those zip codes having sufficient activity
data.head()

Complaint,APPLIANCE,Abandoned Vehicle,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Animal-Abuse,Appliance,Asbestos,BEST/Site Safety,...,Vacant Lot,Vending,Violation of Park Rules,WATER LEAK,Water Conservation,Water Leak,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment
Zip,,,,,,,,,,,,,,,,,,,,,
10000.0,0.000000,0.005464,0.000000,0.000000,0.0,0.049180,0.027322,0.0,0.000000,0.0,...,0.000000,0.027322,0.136612,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
10001.0,0.000000,0.001866,0.003939,0.000000,0.0,0.001451,0.003317,0.0,0.000207,0.0,...,0.000000,0.013270,0.001659,0.0,0.000415,0.0,0.000415,0.004769,0.0,0.0
10002.0,0.000000,0.002056,0.001157,0.000000,0.0,0.001799,0.003213,0.0,0.000129,0.0,...,0.000129,0.001542,0.006940,0.0,0.000000,0.0,0.000771,0.003341,0.0,0.0
10003.0,0.000135,0.002431,0.000946,0.000135,0.0,0.001621,0.006754,0.0,0.001081,0.0,...,0.000000,0.003917,0.005808,0.0,0.000135,0.0,0.002296,0.003377,0.0,0.0
10004.0,0.000000,0.001427,0.000000,0.000000,0.0,0.000713,0.000713,0.0,0.000000,0.0,...,0.000000,0.106277,0.017832,0.0,0.000000,0.0,0.000000,0.000713,0.0,0.0


In [6]:
# Add the income level feature into the dataset

income = pd.read_csv('data/zipcode_income.csv', delimiter=',')
income = income.iloc[:,:2]
income[income.isna()] = 0
income.rename(columns={'ZIPCODE':'Zip', 'median_familyIncome(USD)' : 'income'}, inplace=True)
income=income.loc[(income.Zip>=10000)&(income.Zip<11500)]
income['income'][income['income'] == 0.0] = income['income'].mean() # Replace 0 income with mean
income['income'] = income['income'] / income['income'].sum()
income.head()

,Zip,income
1,10001,0.007785
2,10002,0.002521
3,10003,0.011663
4,10004,0.005682
5,10006,0.012767


In [7]:
# Add the area size feature into the dataset

area = pd.read_csv('data/zips_area.csv', delimiter=',')
area = area.iloc[:,:2]
area.rename(columns={'ZIPCODE':'Zip', 'AREA' : 'area'}, inplace=True)
areas = area.copy()
area=area.loc[(area.Zip>=10000)&(area.Zip<11500)]
area['area'] = area['area'] / area['area'].sum()
area.head()

,Zip,area
0,11436,0.002820
1,11213,0.003681
2,11212,0.005214
3,11225,0.002944
4,11218,0.004580


In [8]:
# Add the population jobs feature into the dataset

jobs = pd.read_csv('data/zipcode_population_Jobs.csv', delimiter=',')
jobs = jobs.iloc[:,:2]
jobs.rename(columns={'ZIPCODE':'Zip', 'totalJobs' : 'jobs'}, inplace=True)
jobs['jobs'] = jobs['jobs'] / areas['area']
jobs.head()

,Zip,jobs
0,10001,0.009204
1,10002,0.000904
2,10003,0.002335
3,10004,0.003356
4,10005,0.001389


In [9]:
# Add the population feature into the dataset

population = pd.read_csv('data/zipcode_population_Jobs.csv', delimiter=',')
population = population[['ZIPCODE','POPULATION']]
population.rename(columns={'ZIPCODE':'Zip', 'POPULATION' : 'population'}, inplace=True)
population['population'][population['population'] < 30.0] = population['population'].mean() # Replace 0 population with mean
population['population'] = population['population'] / areas['area']
population.head()

,Zip,population
0,10001,0.000987
1,10002,0.002744
2,10003,0.001331
3,10004,0.000092
4,10005,0.000220


In [10]:
# Add the house price feature into the dataset

housePrice = pd.read_csv('data/zipcode_housePrice.csv', delimiter=',')
weights = [5000,12500,17500,22500,27500,32500,37500,45000,55000,65000,75000,85000
                    ,95000,112500,137500,162500,187500,225000,275000,350000,450000,625000,875000
                        ,1250000,1750000,2000000]
for i in range(len(weights)):
    housePrice.iloc[i,2:] = housePrice.iloc[i,2:]*weights[i]
housePrice.iloc[:,1] [housePrice.iloc[:,1] == 0] = 1

tmp = (housePrice.iloc[:,2:] != 0 ).sum(axis=1)
tmp[tmp == 0] = 1
housePrice = pd.concat([housePrice.iloc[:,0],housePrice.iloc[:,2:].sum(axis=1) / tmp ], axis = 1 )
housePrice.rename(columns={'ZIPCODE':'Zip', 0: 'house_price'}, inplace=True)
housePrice['house_price'] = housePrice['house_price']/housePrice['house_price'].sum()

In [11]:
data = pd.merge(data, income, on="Zip")
data = data.merge(population)
data = data.merge(jobs)
data = data.merge(housePrice)
data = data.merge(area)
data = data.drop_duplicates(subset=['Zip'], keep='last')
data.head()

,Zip,APPLIANCE,Abandoned Vehicle,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Animal-Abuse,Appliance,Asbestos,...,Water Leak,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment,income,population,jobs,house_price,area
0,10001.0,0.000000,0.001866,0.003939,0.000000,0.0,0.001451,0.003317,0.0,0.000207,...,0.0,0.000415,0.004769,0.0,0.0,0.007785,0.000987,0.009204,0.000437,0.002211
1,10002.0,0.000000,0.002056,0.001157,0.000000,0.0,0.001799,0.003213,0.0,0.000129,...,0.0,0.000771,0.003341,0.0,0.0,0.002521,0.002744,0.000904,0.000571,0.003265
2,10003.0,0.000135,0.002431,0.000946,0.000135,0.0,0.001621,0.006754,0.0,0.001081,...,0.0,0.002296,0.003377,0.0,0.0,0.011663,0.001331,0.002335,0.002566,0.001930
6,10004.0,0.000000,0.001427,0.000000,0.000000,0.0,0.000713,0.000713,0.0,0.000000,...,0.0,0.000000,0.000713,0.0,0.0,0.005682,0.000092,0.003356,0.000000,0.000083
7,10006.0,0.000000,0.007491,0.001873,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.001873,0.0,0.0,0.012767,0.000076,0.000546,0.001680,0.000213


In [12]:
data1 = pd.read_csv('data/zips_merged.csv', delimiter=',')
data1 = data1.rename(columns={'total': 'weight', 'w_zip':'origin', 'h_zip':'destination'})
data1 = data1[data1.destination.isin(data1.origin.unique())]
data1 = data1[['origin', 'destination', 'weight', 'true_label']]
data1 = data1[data1['origin'].isin(data['Zip'])][data1['destination'].isin(data['Zip'])]
data2 = data[data['Zip'].isin(data1['destination'])]

In [13]:
data.head()

,Zip,APPLIANCE,Abandoned Vehicle,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Animal-Abuse,Appliance,Asbestos,...,Water Leak,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment,income,population,jobs,house_price,area
0,10001.0,0.000000,0.001866,0.003939,0.000000,0.0,0.001451,0.003317,0.0,0.000207,...,0.0,0.000415,0.004769,0.0,0.0,0.007785,0.000987,0.009204,0.000437,0.002211
1,10002.0,0.000000,0.002056,0.001157,0.000000,0.0,0.001799,0.003213,0.0,0.000129,...,0.0,0.000771,0.003341,0.0,0.0,0.002521,0.002744,0.000904,0.000571,0.003265
2,10003.0,0.000135,0.002431,0.000946,0.000135,0.0,0.001621,0.006754,0.0,0.001081,...,0.0,0.002296,0.003377,0.0,0.0,0.011663,0.001331,0.002335,0.002566,0.001930
6,10004.0,0.000000,0.001427,0.000000,0.000000,0.0,0.000713,0.000713,0.0,0.000000,...,0.0,0.000000,0.000713,0.0,0.0,0.005682,0.000092,0.003356,0.000000,0.000083
7,10006.0,0.000000,0.007491,0.001873,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.001873,0.0,0.0,0.012767,0.000076,0.000546,0.001680,0.000213


In [14]:
# function for loading data, returns adjacency matrix, initial feature assignments and true labels

def load_data():

    G = nx.from_pandas_edgelist(data1, 'origin', 'destination', 'weight',create_using=nx.DiGraph())
    adj_list = np.array([nx.adjacency_matrix(G).todense()], dtype=float)
    
    init_feat = data.to_numpy()[:,:-5]
    
    true_label = data.to_numpy()[:,-5:]
    
    return adj_list,init_feat,true_label

adj,feature,labels = load_data()

pca = decomposition.PCA(n_components=10)
feature = pca.fit_transform(feature)

features = np.expand_dims(feature, axis=0)


In [15]:
# import packages

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
torch.set_printoptions(sci_mode=False)
import time

In [16]:
# set initial model config

cuda = torch.cuda.is_available()
weight_decay = 1e-8
epochs = 10000
seed = 635
hidden = 16
lr = 0.001

In [17]:
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [18]:
def normalize(adj):

    adj = torch.FloatTensor(adj)
    adj_id = torch.FloatTensor(torch.eye(adj.shape[1]))
    adj_id = adj_id.reshape((1, adj.shape[1], adj.shape[1]))
    adj_id = adj_id.repeat(adj.shape[0], 1, 1)
    adj = adj + adj_id
    rowsum = torch.FloatTensor(adj.sum(2))
    degree_mat_inv_sqrt = torch.diag_embed(torch.float_power(rowsum,-0.5), dim1=-2, dim2=-1).float()
    adj_norm = torch.bmm(torch.transpose(torch.bmm(adj,degree_mat_inv_sqrt),1,2),degree_mat_inv_sqrt)

    return adj_norm

In [19]:
def doublerelu(x):
    return torch.clamp(x, 0, 1)

In [20]:
class GNN1Layer(Module):

    def __init__(self, batch_size, in_features, out_features, first):
        super(GNN1Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size
        
        # Initialse W1 = 1, W2 = 0 as pytorch learnable weights (parameters) that have require_grad = True which is
        # required for calculating gradients while backpropogating using gradient descent
        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        self.weight1 = Parameter(weight1_eye)
        if not first:
            self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))
        else:
            self.weight2 = Parameter(torch.empty(batch_size, in_features, out_features))
            nn.init.kaiming_normal_(self.weight2, mode='fan_out')

    def forward(self, input, adj):
        # first term H*W1
        v1 = torch.bmm(input, self.weight1)
        # second term adj_norm*H*W2
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        # adding the two terms
        output = v1 + v2

        return output

In [21]:
class GNN1(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden, first):
        super(GNN1, self).__init__()

        self.gc1 = GNN1Layer(batch_size, nfeat, hidden, first)
        self.gc2 = GNN1Layer(batch_size, hidden, ndim, first)

    def forward(self, x, adj):

        # Applying activation function sigma (doublerelu) on the layer propogation
        x = doublerelu(self.gc1(x, adj))
        x = doublerelu(self.gc2(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        
        return x

In [22]:
def train(adj,features,labels,train_indices,val_indices,first=False):
    
    # calculate symmetric normalisation for layer propogation
    adj_norm = normalize(adj)
    
    labels = labels - 1
    
    # Convert from numpy to torch tensors
    adj = torch.FloatTensor(adj)
    adj_norm = torch.FloatTensor(adj_norm)
    features = torch.FloatTensor(features)
    labels = torch.FloatTensor(labels)
    
    # initialise the model
    model = GNN1(batch_size=adj.shape[0],
                nfeat=features.shape[-1],
                ndim=5,
                hidden=hidden,
                first=first)
    
    # Transfer the weights to GPU for training
    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()
        labels = labels.cuda()
    
    # Train model
    t_total = time.time()

    # Using adam optimizers for backpropogation
    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)
    
    # loss function criteria is MSE
    criterion = nn.MSELoss()
    
    # Train for the no of epochs
    for epoch in range(epochs):

        t = time.time()
        
        model.train()
        
        # Pytorch accumulates gradient after every operation on tensors (defined by the model architecture)
        # with require_grad = True. With each new epoch, we need to reset this gradient to 0 to calculate gradient
        # for this epoch.
        optimizer.zero_grad()

        # get the output from forward propogation of our model
        output = model(features, adj_norm)
        
        
        # Calculate Train accuracy
        train_output = output[:,train_indices,:]
        train_labels = labels[train_indices,:]
        
        #train_accuracy = torch.sum(torch.argmax(train_output,axis=2)==train_labels.reshape(1,-1))/train_labels.shape[0]
        
        # Calculate the loss between our models training output and true label
        loss = criterion(torch.flatten(output),torch.flatten(labels))
        
        # Calculate the gradients 
        loss.backward(retain_graph=True)

        # Update the weights
        optimizer.step()
        
        model.eval()
        
        # Calculate Validation accuracy
        with torch.no_grad():
            val_output = output[:,val_indices,:]
            val_labels = labels[val_indices,:]
            #val_accuracy = torch.sum(torch.argmax(val_output,axis=2)==val_labels.reshape(1,-1))/val_labels.shape[0]

        # Print summary of training 
        if epoch == 0:
            best_loss = loss
            best_output = output
            #best_acc = train_accuracy
            #best_val_acc = val_accuracy
            best_val_output = val_output
        else:
            if loss < best_loss:
                best_loss = loss
                best_output = output
                #best_acc = train_accuracy
                #best_val_acc = val_accuracy
                best_val_output = val_output

        if epoch == 0 or (epoch+1) % 1000 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  #'Train Accuracy: {:.4f}'.format(best_acc.item()),
                  #'Validation Accuracy: {:.4f}'.format(best_val_acc.item()),
                  'Loss: {:.8f}'.format(best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))
            
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return best_loss,best_output#,best_val_output

In [23]:
# set Train %

train_percentage = .8
    
# Train set
number_of_rows = features[0].shape[0]
train_indices = np.random.choice(number_of_rows, size=int(train_percentage*number_of_rows), replace=False)
val_indices = np.setdiff1d(np.arange(adj.shape[1]),train_indices)

# Start Train
prev_loss, op = train(adj,features,labels,train_indices,val_indices,True)

# Keep training recurrently until the loss stops decreasing
loss, op = train(adj,op.cpu().detach().numpy(),labels,train_indices,val_indices)
while loss < prev_loss :
    prev_loss = loss
    loss, op = train(adj,op.cpu().detach().numpy(),labels,train_indices,val_indices)

Epoch: 0001 Loss: 1.44592106 time: 1.3896s
Epoch: 1000 Loss: 1.41815448 time: 0.0020s
Epoch: 2000 Loss: 1.41812420 time: 0.0020s
Epoch: 3000 Loss: 1.41811037 time: 0.0020s
Epoch: 4000 Loss: 1.41810071 time: 0.0020s
Epoch: 5000 Loss: 1.41809499 time: 0.0020s
Epoch: 6000 Loss: 1.41808975 time: 0.0020s
Epoch: 7000 Loss: 1.41808832 time: 0.0020s
Epoch: 8000 Loss: 1.41808701 time: 0.0020s
Epoch: 9000 Loss: 1.41808319 time: 0.0030s
Epoch: 10000 Loss: 1.41807950 time: 0.0030s
Optimization Finished!
Total time elapsed: 21.6084s
Epoch: 0001 Loss: 1.41807950 time: 0.0020s
Epoch: 1000 Loss: 1.41657972 time: 0.0020s
Epoch: 2000 Loss: 1.41656423 time: 0.0020s
Epoch: 3000 Loss: 1.41656339 time: 0.0020s
Epoch: 4000 Loss: 1.41656291 time: 0.0020s
Epoch: 5000 Loss: 1.41656268 time: 0.0020s
Epoch: 6000 Loss: 1.41656244 time: 0.0020s
Epoch: 7000 Loss: 1.41656196 time: 0.0020s
Epoch: 8000 Loss: 1.41656160 time: 0.0020s
Epoch: 9000 Loss: 1.41656125 time: 0.0020s
Epoch: 10000 Loss: 1.41656089 time: 0.0020s


KeyboardInterrupt: 

In [24]:
op[0]

tensor([[    0.2007,     0.2073,     0.1980,     0.1914,     0.2026],
        [    0.2021,     0.2017,     0.2004,     0.1959,     0.1999],
        [    0.2000,     0.2064,     0.1990,     0.1950,     0.1997],
        [    0.2003,     0.2075,     0.1999,     0.1968,     0.1955],
        [    0.1997,     0.2070,     0.1987,     0.1948,     0.1997],
        [    0.2001,     0.2062,     0.1990,     0.1950,     0.1997],
        [    0.2002,     0.2059,     0.1991,     0.1951,     0.1997],
        [    0.2006,     0.2015,     0.1983,     0.2001,     0.1995],
        [    0.1999,     0.2054,     0.1984,     0.1966,     0.1997],
        [    0.2001,     0.2037,     0.1983,     0.1984,     0.1995],
        [    0.2001,     0.2034,     0.1981,     0.1990,     0.1995],
        [    0.2015,     0.1990,     0.1984,     0.2013,     0.1999],
        [    0.2001,     0.2038,     0.1982,     0.1981,     0.1997],
        [    0.2004,     0.2016,     0.1978,     0.2005,     0.1997],
        [    0.2000,

In [25]:
labels

array([[7.78543537e-03, 9.87387474e-04, 9.20350151e-03, 4.36505470e-04,
        2.21058644e-03],
       [2.52145888e-03, 2.74391643e-03, 9.04086801e-04, 5.71425342e-04,
        3.26466362e-03],
       [1.16634125e-02, 1.33131281e-03, 2.33533682e-03, 2.56631203e-03,
        1.93026346e-03],
       [5.68181818e-03, 9.22838151e-05, 3.35610116e-03, 0.00000000e+00,
        8.33190778e-05],
       [1.27673068e-02, 7.64046460e-05, 5.46048348e-04, 1.68026261e-03,
        2.13250676e-04],
       [1.01689799e-02, 1.74345775e-04, 1.04462237e-03, 2.90814683e-03,
        6.61857743e-04],
       [3.76397986e-03, 1.28333307e-03, 1.86459701e-04, 1.63264383e-03,
        1.97562355e-03],
       [1.54275763e-02, 5.98497357e-04, 1.67910145e-03, 4.86391809e-03,
        1.21348439e-03],
       [1.25772628e-02, 1.16160683e-03, 1.96591980e-03, 7.18091180e-03,
        2.25077044e-03],
       [1.07580802e-02, 2.70782142e-04, 5.38284915e-04, 4.43875043e-03,
        1.14209735e-03],
       [9.81412160e-03, 1.2990